# Criptografia pós-quântica -> Dilithium

## Descrição do Problema

Este problema é dedicado às candidaturas finalistas ao concurso NIST Post-Quantum Cryptography na categoria de assinatura digital. 

    1. O objetivo deste trabalho é a criação de protótipos em Sagemath para o algoritmo Dilithium.
    2. Neste protótipo é implementada a versão 5++.

## Abordagem

Este código, implementado em Python com a biblioteca Sagemath, define uma implementação de um protótipo do esquema de assinatura digital Dilithium. O algoritmo Dilithium, é um esquema de assinatura digital pós-quântica baseado na dificuldade em encontrar vetores curtos em reticulados, que segue uma abordagem de Modular LWE e cujo design do esquema é baseado na abordagem Fiat-Shamir with Aborts(Lyubashevsky).

O esquema Dilithium implementado consiste em três algoritmos principais: Keygen, Sign e Verify.

O código também inclui várias funções auxiliares para realizar operações como a transformação de strings em bits, a aplicação de funções de hash específicas e a geração de matrizes aleatórias de polinômios. Além disso, ele também implementa funções para decompor polinômios e calcular a norma infinita.

In [7]:
# Métodos auxiliares

from hashlib import shake_128, shake_256, sha256, sha512

Zx.<x>= ZZ[]
Zq.<z>= PolynomialRing(GF(8380417))
Rq.<z>= Zq.quotient(z^256 + 1)
R.<x>= Zx.quotient(x^256 + 1)

# Converte uma string para binário
def str_to_bits(msg, print_msg=False):
    bits = ""

    for char in msg:
        byte = ord(char)
        bits += "{0:b}".format(byte).zfill(8)
    
    if print_msg:
        print("\"{0}\" => {1}\n".format(msg, bits))
        
    return bits


# Implementa um XOF (SHAKE-256)
def XOF(p, i):
    return shake_256(str(i).encode() + str(p).encode()).digest(int(2000))

#Calcula o digest dos valores 'a' e 'b' utilizando SHA-512
def G(a, b=""):
    digest = sha512(str(a).encode() + str(b).encode() ).digest()
    return digest[:32], digest[32:]


# Constrói a matriz A
def parse(b, q, n):
    i = 0
    j = 0
    a = []
    
    while j < n and i + 2 < len(b):
        d1 = b[i] + 256 * b[i + 1] % 16
        d2 = b[i+1] // 16 + 16 * b[i + 2]
        
        if d1 < q:
            a.append(d1)
            j += 1
        
        elif d2 < q and j < n:
            a.append(d2)
            j += 1
        
        i += 3
    
    return a

### Métodos necessários na classe Dilithium 

#### KeyGen (Geração de Chaves)

O método keygen é responsável por gerar um par de chaves - uma chave pública e uma chave privada - que são usadas para a assinatura digital e a verificação da assinatura, respectivamente.

    1. Inicia com a geração de um elemento aleatório d no anel Rq.

    2. Calcula a função G em d, que é usada para obter a "seed" p.

    3. Através da "seed" p e da função XOF, gera a matriz A que é composta por polinômios no anel Rq.

    4. Em seguida, gera duas matrizes de polinômios s1 e s2.

    5. A chave pública é então calculada como pk = (p, t), onde t = A * s1 + s2. A chave privada é sk = (s1, s2).

    6. Finalmente, retorna a chave pública e privada.

O que foi feito aqui é um tipo de otimização de espaço. No Dilithium, a matriz A é gerada a partir de uma seed, a qual é gerada aleatoriamente. O fato é que essa matriz é muito grande, e guardá-la completamente em memória é bastante custoso em termos de espaço. No entanto, como ela é determinística (ou seja, a mesma seed sempre produzirá a mesma matriz A), é possível simplesmente guardar a seed e gerar A sempre que necessário.

Assim, ao invés de guardar a matriz A na chave pública e privada, optou-se por guardar apenas a seed (ρ). Quando necessário, a matriz A é recriada a partir da seed. Isto é feito em cada operação que necessita de A, como a geração da chave, a assinatura e a verificação da assinatura. Este processo economiza uma quantidade significativa de espaço, já que uma seed é muito menor do que a matriz A.

Com isto, a chave pública passa a ser composta pela seed (ρ) e por t, enquanto a chave privada passa a ser composta por s1 e s2. A seed é usada para gerar a matriz A sempre que necessário.


#### Sign (Assinatura)

O método Sign é utilizado para gerar uma assinatura digital para uma determinada mensagem. A assinatura é uma prova de que a mensagem foi assinada por alguém que detém a chave privada correspondente à chave pública dada.

    1. Começa por extrair a "seed" p e a matriz t da chave pública, e as matrizes s1 e s2 da chave privada.

    2. A matriz A é reconstruída a partir da "seed" p usando a função XOF.

    3. Em seguida, entra num loop onde gera um vetor polinomial y com coeficientes aleatórios e calcula ay = A * y e w = high_bits(ay), sendo high_bits uma função que mantém apenas os bits mais significativos de ay.

    4. Calcula a função hash c da mensagem concatenada com a string representativa de w.

    5. O vetor polinomial z é então calculado como z = y + c * s1.

    6. Se z e ay - c * s2 satisfizerem certas condições de norma, então a assinatura é (z, c) e o loop termina. Caso contrário, o processo é repetido até que uma assinatura válida seja encontrada.

    7. Por fim, retorna a assinatura (z, c).

O método sign garante que, dada uma mensagem e uma chave privada, é possível gerar uma assinatura que possa ser verificada por qualquer pessoa que possua a chave pública correspondente.

#### Verify (Verificação)

O método Verify é utilizado para verificar a autenticidade de uma assinatura digital para uma determinada mensagem utilizando a chave pública correspondente. O método assegura que a assinatura foi efetivamente gerada por alguém que detém a chave privada correspondente.

    1. Primeiro, extrai a "seed" p e a matriz t da chave pública.

    2. A matriz A é então reconstruída a partir da "seed" p utilizando a função XOF.

    3. A assinatura, que é um par (z, c), é separada nas suas componentes. Aqui, z é um vetor polinomial e c é a hash da mensagem.

    4. Em seguida, calcula az = A * z e w = high_bits(az - c * t), onde high_bits é uma função que mantém apenas os bits mais significativos.

    5. A hash c_ da mensagem concatenada com a string representativa de w é então calculada.

    6. Se z satisfaz uma certa condição de norma e c é igual a c_, a verificação é bem sucedida e retorna True. Caso contrário, retorna False.

O método verify garante que, dada uma mensagem, uma assinatura e uma chave pública, é possível verificar se a assinatura é válida e, portanto, se a mensagem é autêntica.

In [8]:
class Dilithium:
    
    def __init__(self):
        # parâmetros versão 5++
        self.n = 256
        self.q = 2^23-2^13+1
        self.h = 60
        self.r = 1753
        self.k = 10
        self.l = 9
        self.gamma1 = 524288
        self.gamma2 = 261888
        self.eta = 2
        self.beta = 120
        
    
    def keygen(self):
        d = Rq.random_element()
        p, _ = G(d)
        
        mat = []
        for i in range(self.k * self.l):
            mat.append(Rq(parse(XOF(p, i), self.q, self.n)))
            
        A = matrix(Rq, self.k, self.l, mat)
      
        S1 = self.sample_array(self.eta, self.l)
        S2 = self.sample_array(self.eta, self.k) 
        
        t = A * S1 + S2
        self.pk = p, t
        self.sk = S1, S2
        
        return self.pk, self.sk
    
    
    def sign(self, m):
        p, t = self.pk
        s1, s2 = self.sk
        
        mat = []
        for i in range(self.k * self.l):
            mat.append(Rq(parse(XOF(p, i), self.q, self.n)))
        A = matrix(Rq, self.k, self.l, mat)
        
        while True:
            y = self.sample_array(self.gamma1 - 1, self.l)
            ay = A * y
            w = self.high_bits(ay)
            
            c = self.hashes(m + str(w))
            cq = Rq(c)
            z = y + cq * s1
            
            az = A * z

            try:
                if self.infinity_norm(z) >= self.gamma1 - self.beta and self.infinity_norm(self.low_bits(ay - cq * s2)) >= self.gamma2 - self.beta:
                    continue
                else:
                    break
            except:
                continue
                
        return z, c
        
        
    def verify(self, pk, m, sig):        
        p, t = pk
        
        mat = []
        for i in range(self.k * self.l):
            mat.append(Rq(parse(XOF(p, i), self.q, self.n)))
        A = matrix(Rq, self.k, self.l, mat)

        z, c = sig
        cq = Rq(c)
        az = A * z
        w = self.high_bits(az - cq * t)
        
        c_ = self.hashes(m + str(w))
        
        if self.infinity_norm(z) < self.gamma1 - self.beta and c_ == c:
            return True
        else:
            return False

        
    # Função de decompose, seguindo a documentação
    def decompose(self, c, alfa):
        r = mod(c, self.q)
        r0 = int(mod(r, int(alfa))) 
        if r0 > alfa / 2:
            r0 = r0 - int(alfa)
            
        if r - r0 == self.q - 1:
            r1 = 0
            r0 = r0 - 1
        else:
            r1 = (r - r0) / (int(alfa))
        return r1, r0

    
    # Função auxiliar de high_bits
    def high_bits_aux(self, c):
        x, _ = self.decompose(c, 2 * self.gamma2)
        return x
   

    # Função auxiliar de low_bits
    def low_bits_aux(self, c):
        _, x = self.decompose(c, 2 * self.gamma2)
        return x
 

    # Função high_bits, seguindo a documentação
    def high_bits(self, pol):
        k = pol.list()
        for i in range(len(k)):
            h = k[i]
            h = h.list()
            
            for j in range(len(h)):
                h[j] = self.high_bits_aux(int(h[j]))
                
            k[i] = h
         
        return k
  

    # Função low_bits, seguindo a documentação
    def low_bits(self, pol):
        k = pol.list()
        for i in range(len(k)):
            h = k[i]
            h = h.list()
            
            for j in range(len(h)):
                h[j] = self.low_bits_aux(int(h[j]))
                
            k[i] = h

        return k
 

    # Gera um array de polinómios
    def sample_array(self, lim, tam):
        S = []
        for i in range(tam):
            pol = []
        
            for j in range(self.n):
                pol.append(randint(1, lim))
                
            S.append(Rq(pol))
        
        S = matrix(Rq, tam, 1, S)
        
        return S

    
    # Calcula Hash e transforma numa lista de coeficientes
    def hashes(self, val):
        seed = XOF(val, "")
        vals =  ''.join([bin(byte)[2:] for byte in seed])

        H = []
        contador = 0
        contador_num = 0
        
        for i in range(0, self.n, 2):
            u = vals[i] + vals[i + 1]
            contador += 1
            
            if u == '11':
                H.append(0)
                contador_num += 1
            elif u == '01':
                H.append(1)
                contador_num += 1
            elif u == '10':
                H.append(-1)
                contador_num += 1
            
            if contador_num >= self.h:
                break
        
        for i in range(self.n - contador_num):
            H.append(0)
        
        return H

    
    # Função auxiliar de infinity_norm
    def infinity_norm_aux(self, pol, n):
        J = pol.list()
        
        for i in range(len(J)):
            k = J[i]
            K = k.list()
            
            for j in range(len(K)):
                K[j] = abs(int(K[j]))
            
            J[i] = K
        
        L = []
        for i in range(len(J)):
            L.append(max(J[i]))
            
        return max(L)
    
    
    # Calcula a norma infinita
    def infinity_norm(self, vetor): 
        res = []
        for i in range(vetor.nrows()):
            norm = self.infinity_norm_aux(vetor[i], self.q)
            res.append(norm)
        
        return int(max(res))
    
    
    # Função power2round, seguindo a documentação
    def power2round(self, w, n):
        r = (n - 1)//2
        
        return Zx(map(lambda x: lift(x + r) - r, w.list()))

### Exemplo de execução

In [9]:
# Cria uma instância da classe Dilithium
d = Dilithium()

# Gera um par de chaves
pk, sk = d.keygen()

# Define uma mensagem
mensagem = "Unidade curricular de Estruturas Criptográficas!"

# Assina a mensagem com a chave privada
assinatura = d.sign(mensagem)

# Verifica a assinatura com a chave pública
verificacao = d.verify(pk, mensagem, assinatura)

# Imprime o resultado da verificação
if verificacao:
    print("A assinatura é válida.")
else:
    print("A assinatura é inválida.")
    
print("\nMensagem definida:", mensagem)
print("\nChave pública:", pk)
print("\nChave secreta:", sk)

A assinatura é válida.

Mensagem definida: Unidade curricular de Estruturas Criptográficas!

Chave pública: (b'n\x92\x991\x88\x82\x97\x97\xe0r\xc0)\x9d\xe2\xf2"\xff\x98\xd2vt\xe0\xb3\x05|%\x7f\xf8w\xe9L)', [    444385*z^255 + 441492*z^254 + 437972*z^253 + 434184*z^252 + 432262*z^251 + 423971*z^250 + 424114*z^249 + 419402*z^248 + 420138*z^247 + 415086*z^246 + 409686*z^245 + 408622*z^244 + 404568*z^243 + 400890*z^242 + 398131*z^241 + 398293*z^240 + 391010*z^239 + 385564*z^238 + 381092*z^237 + 380720*z^236 + 377936*z^235 + 376180*z^234 + 374639*z^233 + 368365*z^232 + 365077*z^231 + 359286*z^230 + 360426*z^229 + 353701*z^228 + 353656*z^227 + 346607*z^226 + 348595*z^225 + 342893*z^224 + 340485*z^223 + 332928*z^222 + 330871*z^221 + 327581*z^220 + 318710*z^219 + 319049*z^218 + 314132*z^217 + 311592*z^216 + 307304*z^215 + 309397*z^214 + 307003*z^213 + 301075*z^212 + 298984*z^211 + 293922*z^210 + 288884*z^209 + 280278*z^208 + 278734*z^207 + 275235*z^206 + 274411*z^205 + 267090*z^204 + 269489*z^